In [ ]:
!pip install -q -U google-generative-ai

import google.generativeai as genai
from google.colab import userdata
import os
import getpass
import json
import time

try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
except:
    print("Please enter your Google API Key:")
    GOOGLE_API_KEY = getpass.getpass("API Key: ")

genai.configure(api_key=GOOGLE_API_KEY)

print("\n🔍 Scanning for available models...")
available_models = [m.name for m in genai.list_models()]
target_model = ""

if 'models/gemini-1.5-flash' in available_models:
    target_model = 'gemini-1.5-flash'
elif 'models/gemini-1.5-flash-001' in available_models:
    target_model = 'gemini-1.5-flash-001'
elif 'models/gemini-pro' in available_models:
    target_model = 'gemini-pro'
else:
    for m in available_models:
        if 'generateContent' in genai.get_model(m).supported_generation_methods:
            target_model = m
            break

if not target_model:
    raise ValueError("❌ No text generation models found for this API key.")

print(f"✅ Selected Model: {target_model}")
model = genai.GenerativeModel(target_model)

In [ ]:
class MedicalTools:
    def __init__(self):
        self.ehr_docs = [
            "Patient: John Doe | DOB: 1980-05-12",
            "Visit 2023-01-10: Diagnosed with Type 2 Diabetes. Prescribed Metformin.",
            "Visit 2023-04-15: Patient reports severe GI distress with Metformin. Discontinued.",
            "Visit 2023-04-20: BMI recorded at 32.5. A1C is 8.4%.",
            "Visit 2023-05-01: Doctor recommends starting Ozempic (Semaglutide)."
        ]

    def search_ehr(self, query):
        print(f"   🔎 [Tool] Searching EHR for: '{query}'...")
        results = [doc for doc in self.ehr_docs if any(q.lower() in doc.lower() for q in query.split())]
        if not results:
            return "No records found."
        return "\n".join(results)

    def submit_prior_auth(self, drug_name, justification):
        print(f"   📤 [Tool] Submitting claim for {drug_name}...")
        justification_lower = justification.lower()
        if "metformin" in justification_lower and ("discontinued" in justification_lower or "intolerance" in justification_lower):
            if "bmi" in justification_lower and "32" in justification_lower:
                return "SUCCESS: Authorization Approved. Auth ID: #998877"
        return "DENIED: Policy requires proof of (1) Metformin failure and (2) BMI > 30."

In [ ]:
class AgenticSystem:
    def __init__(self, model, tools):
        self.model = model
        self.tools = tools
        self.history = []
        self.max_steps = 6

        self.system_prompt = """
        You are an expert Medical Prior Authorization Agent.
        Your goal is to get approval for a medical procedure/drug.

        You have access to these tools:
        1. search_ehr(query)
        2. submit_prior_auth(drug_name, justification)

        RULES:
        1. ALWAYS think before you act.
        2. You MUST output your response in STRICT JSON format:
           {
             "thought": "Your reasoning here",
             "action": "tool_name_or_finish",
             "action_input": "argument_string_or_dict"
           }
        3. Do not guess patient data. Use 'search_ehr'.
        4. If you have the evidence, use 'submit_prior_auth'.
        5. If the task is done, use action "finish".
        """

In [ ]:
    def execute_tool(self, action_name, action_input):
        if action_name == "search_ehr":
            return self.tools.search_ehr(action_input)
        elif action_name == "submit_prior_auth":
            if isinstance(action_input, str):
                return "Error: submit_prior_auth requires a dictionary."
            return self.tools.submit_prior_auth(**action_input)
        else:
            return "Error: Unknown tool."

    def run(self, objective):
        print(f"🤖 AGENT STARTING. Objective: {objective}\n" + "-"*50)
        self.history.append(f"User: {objective}")

        for i in range(self.max_steps):
            print(f"\n🔄 STEP {i+1}")
            prompt = self.system_prompt + "\n\nHistory:\n" + "\n".join(self.history) + "\n\nNext JSON:"

            try:
                response = self.model.generate_content(prompt)
                text_response = response.text.strip().replace("```json", "").replace("```", "")
                agent_decision = json.loads(text_response)
            except Exception as e:
                print(f"   ⚠️ Error parsing AI response. Retrying... ({e})")
                continue

            print(f"   🧠 THOUGHT: {agent_decision['thought']}")
            print(f"   👉 ACTION: {agent_decision['action']}")

            if agent_decision['action'] == "finish":
                print(f"\n✅ TASK COMPLETED: {agent_decision['action_input']}")
                break

            tool_result = self.execute_tool(agent_decision['action'], agent_decision['action_input'])
            print(f"   👁️ OBSERVATION: {tool_result}")

            self.history.append(f"Assistant: {text_response}")
            self.history.append(f"System: {tool_result}")

            if "SUCCESS" in str(tool_result):
                print("\n🎉 SUCCESS! The Agent successfully navigated the insurance portal.")
                break

In [3]:
tools_instance = MedicalTools()
agent = AgenticSystem(model, tools_instance)
agent.run("Please get prior authorization for Ozempic for patient John Doe.")

🤖 AGENT STARTING. Objective: Please get prior authorization for Ozempic for patient John Doe.
--------------------------------------------------

🔄 STEP 1
   🧠 THOUGHT: The user wants to get prior authorization for Ozempic for John Doe. To do this, I need to find the medical justification from John Doe's EHR. I will start by searching his EHR for relevant information.
   👉 ACTION: search_ehr
   🔎 [Tool] Searching EHR for: 'John Doe Ozempic justification'...
   👁️ OBSERVATION: Patient: John Doe | DOB: 1980-05-12
Visit 2023-05-01: Doctor recommends starting Ozempic (Semaglutide).

🔄 STEP 2
   🧠 THOUGHT: The previous EHR search indicated that the doctor recommended Ozempic, but it didn't provide a medical justification. I need to find the specific diagnosis or condition that warrants Ozempic for John Doe. I will search the EHR for his medical conditions, specifically looking for information related to diabetes or weight management, as Ozempic is commonly prescribed for these.
   👉 ACTION: